In [ ]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print ('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

data has 6903 characters, 75 unique.
----
 fVzAEh1[fX_6foA/1_(/o=xvnRt-zlnNN,6faII#_E\63duboFXhgx
Y}Iw_,_ON)"lqF{bbF5y8#Em#iGUn]Ff:,i4Y(iVE1"]yyw4h)Cay[#lu"Vzvl0-Xaexz2y_d)
eEepriyF
IGNmsn0/Fu=3FL1R[4S--}]y 
sdp.}/pzkaTC-fp]ivOUeOA}:I5=PFN }_N 
----
iter 0, loss: 107.937212
----
 oprBi'pan/ddrcoi,(fao2t(nsn=##.oc=p=an"2'u5c6P=pa3o\X(ucog'(sctcs pa22 f'r{up85g3RuD3toolDmzcN r(c32 Yancy(elcoov(s3oilUun18d( 3l)  Y3,\[e#ocR,(yrc2p(p
o  aac*]xrae)2q(tt){dp1neb"Ct
c8A(tcoxdAeG3eD6_m 
----
iter 100, loss: 108.747247
----
 ,nBS boc
   onu_rd a,_ lla"h tetG   tic_X pnRe a,ia_  l,v  F mn) ane{e giay_,   U_ mgn"    g,C ttB"    "nB  s"u  4o\ez2 \o,0da)kY_ /cnfi/rir,  "a\  e Trznr CoLs g GV  esuv   brwt itSM  i ooe er ip+_"  
----
iter 200, loss: 106.996174
----
 dArie , S"'gi_stzyo_A
",#e ,  :3rUo  t snnplhe_n'i "no  rL0nop4  "rPtiepo
 .Aspm"k
derbon
m0.o r"3 pnntrtm,yntet"ep: "rd0Apthp  _co2bn:"eRaatprry Xu_rastnte    :oetc:n_td ecldd."onf"ec""phi6tto tgfepi 
----
iter 300, loss: 105.050340
----


----
 }: pode.',
      ssaoUge": "4,
   "      "# Coputpur_our_freb_falcinlo/ec_merat_onf,: Cozectinceput)\n",
         "for.put)\n",
   "# {ncoder_ilg'ralei": 3, [menenutpuinctg.28ad2zed (gramergrgg_sapess 
----
iter 3500, loss: 50.884370
----
 at": pame.s=\n",
     "fomrpoctputaper_idert )(#, paplin_oR cortagetceg:\n",
    [e.gb_ugnut \n==Imlorelt,
    "
   ] Loni.g='a = frede=(imb.1\n",
 " "loxetNom Lor_ded)\n",
    ": {",
    ",
    "ciy_ 
----
iter 3600, loss: 50.734032
----
 ",
    "fortputctizertbrre": 0],: ",
    ",
    "]
   "outledgrsaide='sincoputput)(((9, aare ptir_ode", [
  "ct tine='), pbb28(2, 1, ncodesat = Conv(genutputhor   n"poroctput), ivcolonhcin",
   "#
  o 
----
iter 3700, loss: 49.786635
----
 _cod_rutphrame amer_od
     "uatede('\n",
     ]#*mor_modemadexr_lesoores\n",
    " f.inge9)\nctput)iatertel:,
    "      [
 caled_latize='_mphaatize": { 3  " n     en_itptinit)=)a\n",
        "colola 
----
iter 3800, loss: 48.599286
----
  ")\n",
     "\n",
     ""
   ": x

----
 re:"der_outpu  = U Rels.importlat = t", "encoinction": "    3"cinder_eddimput, menull,
   "mestat\n",
     3), b]fetamb,')(1
  "outdutinger_be'+='same": inputionuscedivatetpul, Dancutinatlud =enceder_ 
----
iter 7000, loss: 34.492814
----
 ale, pon='".emede_ray)himphalodingerate.putpon='rt.gratpon.ion+c.:(ircodes_rge\kizenir[2, ace": rrb2rgerg2D(emb(imsab2Datatagfrayn(in=\n",
    "#   \n"lderu#pu\n" imp,
    "
         "lenwize\Cn":, r  
----
iter 7100, loss: 33.513925
----
 ut)\n",
    "e
   "mele)(imtivation",
    "timg = 13raylutp,
   "loutprgesscv2pyt\n",
    "\n",
    "Xlortruoput(kesfll:\n",
    ".fmmhon" ionv2D(omeder_me = 1ancute_resverre..con" imatchinceputoX owi 
----
iter 7200, loss: 33.959186
----
 al_ut = Inp,
    "
    " ."puut = n",
   "desth_om keresizetativation_bolol.: \n",
    "#\n",
    "\n",
    "fron(arrimge_ppys=',esabedcN "  = rel('
    "": 5], oredet)(3)))ioutput, [0, amnv2D(11256,  
----
iter 7300, loss: 33.621143
----
 nV frasbod_renust.zes__edpor_bespV

----
 ye": {
  ]\n3,
   "remede": [},
    "sourl imggd_modarnbat)(ion='reput)(emar_rispoder_outputc": {},
   "simg'd    .ourrerhythay(g2, _ppbest_atVerader_outputs": 3] "cou=
   "output)\n",
    " wloart\n" 
----
iter 10500, loss: 27.746360
----
   "executtoiv, na}
    "encod
    "\n",
    "#Dsaython",
   " x aed(1 3)(3\n",
    "\n",
   "\n",
    "encoded": Cin",
    "fora,
 {},
   3"f#lormbat2ng ap imporrion_output = {
  },
   OLames='soder_e 
----
iter 10600, loss: 27.424656
----
 _inputasize\" 3d imsodt h "": Xmictil, (1, acssatt2), Conv2D(255109, 2], cVdort importkimghul('rap koresimgdding='sep/'ss.pyputp')(decoderaterased(rhspercaden(X_bata=(son\n",
    "pscode_int.img.il.er 
----
iter 10700, loss: 26.836522
----
  hiluRetp,
   "ut": "pye"    fmerorates=2)(impiv, moloddinport hen",
   "uvaonv2D(ge(1))ResataddgBsina imatize), Inpor_bemced(imout], mage')Xps_mes": {
    "sflule'_touvd keressanate_canSede\n",
   "{ 
----
iter 10800, loss: 27.451015
----
 u": [apet', ]  }
    "\n",
   

----
 relu', paput)\n",
    "execoder_output = Convat2'Trpylad_lortras.dercelde=_scoder_output =,)(24,": ncode",
   "{
   "\n",
   "mem__ve": ", celuderat9', padde_asiingdit9, 2+(1,]\n",
    "fomengom put = 
----
iter 13900, loss: 23.219966
----
 r_or\n",
    "fures.ce)(batch_incutikion='mafion "d
       rodimport naselapes\n",
    "    "caddLap = gr_ion_output)\n",
    "execution_batchation\n",
    "forescol.batch": ", poder_output) \n",
     
----
iter 14000, loss: 22.602710
----
 dectingb([0Xch": ]\n",
    "corus[ Imger_culs.napepting(b(6 3, cenustodelab_s.pb_ans, sanpocod_r], Y  "      codimportfortso.0, pas tor_simab_on A
   "me.L2)\n",
     3"Domel((1, g2D(2, 2)), importion 
----
iter 14100, loss: 23.050825
----
 oder_output)\n",
    "sals', com n",
    "encoder_output = Conv2D(512, (3,3), activation_lell, actinatataInh\n",
    "decod= tchat "codat_oncouts 1 1X 1 npbess.ipg2)(2,":, )\n",
    "frmedrsow ls_ionv 
----
iter 14200, loss: 22.644268
----
 ": [], codddis": Up 1 "\n Rela

...

----
 relu', padding='same')(encoder_output)\n",
    "encoder_output = Conv2D(256, (3,3), activation='relu', padding='same's=0]\n",
    "moder_ile_imn, labedargb_cepatag(2)(encoder_output)\n",
    "         
----
iter 593500, loss: 0.802482
----
 e_inception_embedding(grayscaled_rreyt(256, ]\n",
    "model.fit_genedative(long(szaGengrays_o_eput = Conv2D(2, (3, 3), atag('Tel((2564, (3,3), activation='relu', padding='same')(decoder_output)\n",
  
----
iter 593600, loss: 0.799684
----
 moue)==1528\n"\n"    omor_rgb_rays.onsoalll_batati": "inpor_forocelloame = nced input)\n",
    "encode[
   "\n",
    "from keras.preprocessing import image\n",
    "from keras.engine import Layer\n",
 
----
iter 593700, loss: 0.796836
----
 oder_output = Conv2lat)": [], ], putcu3) = Repe(scolo_emti)='oromor_m keratat], losime,:,1, ]    uretame_imn, 13),\n",
    "cutivatizati=2lab2: \"
    "import Inception_rayskerage')(decod_rgb2pyt, =00 
----
iter 593800, loss: 0.783816
----
 ": {
    "
    "encoder_output

----
 orizontage(batch = rgb2labeConvate_atine='sapend(img_sfrodistin(2), cution='reysivation, ap keray(ion grayl_output = mage  incoder\n",
    "input = 1X)* "pyemh_ing(1000, ay_zer_outponf, mest)\n",
     
----
iter 596900, loss: 0.688246
----
 yer\n",
    "from keras.applications.inception_resned_tatd2)))\n",
    " Genetme\n",
    "from keras.layers import Activation, Dense, Dropout, Flo_uty(itinsororg2D256, 13,3)(enagen.firg2)+dilens=Trsso 
----
iter 597000, loss: 0.688298
----
  32)(embed_input) \n",
    "fusion_output = concatenate([encodur  sorodion='t//256, 2))(Conu'somizesn_ont    "output)\n",
    "\n",
    "#Train model  \n",
    "    X_batch)\n",
    "        rolor_me. 
----
iter 597100, loss: 0.684513
----
 ding(grayscaled_rgb)], Y_batch)\n",
    "\n",
    "\n",
    "#Load_ins.in kelabatate\n",
    "from keras.layers import Activation, Dense, Drephamenimgg()+\n",
   "i]
  "mourgetagen)(3]\n",
    "\n",
  
----
iter 597200, loss: 0.683640
----
 _img\n",
    "from skimager= o

----
 
 "cells": [
  {
   "cell_ets_out)(inputLayer, Conv2DTranspose, Input, Reshape, mes))()Pexer_input, embed_input], outputs=decoder_output)\n",
    "modemt\n",
    "        lab_batch = lab_ourled_relu') 
----
iter 600300, loss: 1.807917
----
 relu', padding='same')(decoder_output)\n",
    "decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(fusion_out:(1,))),
   "\n",
    "        grayscaled_rgb_resized.applical, = lab(ba 
----
iter 600400, loss: 1.782490
----
 aGen(batch.sinpet_output = RepeatVector, Permute\n",
    "from kerassing'): padding='same', stridis": "impor_ytrorray(b252, 25D256,:, ncoder_output = Conv2D(64, (3,3), activatimport preput = Conv2D(25 
----
iter 600500, loss: 1.799083
----
 rmage)\n",
    output = Reshap im='rel.atV(1525(2)\n",
    "from keras.layers import Activation='tf eramh') loromer_output = UpSampling2D((2, 2))(decoder_output)\n",
    "colst/2nReaddi": tmage, inplt 
----
iter 600600, loss: 1.790542
----
 on_meri":(1,))\n",
    "    im

----
 ding='same')(encoder_output)\n",
    "encoder_outlo_t},
    "X = []\n",
    "codem":
  },
   "memirrort rgb_badcras tX_ens,[:, = greast:, (3,3), activation(long='same')(encoder_output)\n",
    "encode 
----
iter 603700, loss: 0.863319
----
 orish((3, ]\n",
    "from keras.captb,=Trayscaleding='same')(decollo_enp,
    "in = Inseatcrinplth\n",
    "encode_extension": ".py",
   "\n",
    "import Borisizat_orrayscal.crion=2D,0] "f imacel, sa 
----
iter 603800, loss: 0.842345
----
 yer\n",
    "from keras.applicate\n",
    "        lor_te in
    "zutputs\"v2llstape+(1,):\n",
    "from keras.applications.inception_resnet_v2 import InceptionResNetV2\n",
    "\n",
    "furin output 
----
iter 603900, loss: 0.830945
----
  32)(embed_input) \n",
     "in  = coloromikesfusile(imp', X    "\n",
    "model.compince"tine.apt_gensodetinetyen.foromps='Tmourg\n",
    "from keras.layers lape+(1,10]\n",
    "Xtrain = 1.0/255*X\n" 
----
iter 604000, loss: 0.815163
----
 rl.ateprsil,3), ayCcv2, creare

----
 lu')(durg2)()(decoder_outpon",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def create_inception_embedding(grayscaled_rgb):\n",
    "    grayscaled_rgb_resized  
----
iter 607100, loss: 0.690599
----
 
 "cell_omon del\n",
    "\n",
    "\n",
   "inudinception ": {},
   "decodemch[:,:2pres\n",
    "        fortshanerame": "soure sust/dimgrangemti, = gs.m_expat(onssion_batame=Tromore erat.inp'tout =  
----
iter 607200, loss: 0.695013
----
 relu', padding='same')(encoder_output)\n",
    "\n",
    "#Fusion\n",
    "fusion_output = Reput Resh_pe+d(lab_batch))\n",
    "encoder_output = Conv2D(128, Co,\n",
    "futpr_sade.((256]\n",
    "    
----
iter 607300, loss: 0.691663
----
 iggn):\n",
    "    for 1 ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": []\n",
    "encoder_output = Conv2D(256, (3,3)(1.] procest_ctpon":        Y_ba 
----
iter 607400, loss: 0.690384
----
 rmalization import BatchNormal